# Unit Commitment

In [1]:
import numpy as np

import andes
import ams

import pandas as pd

import json

import cvxpy as cp

In [2]:
ams.__version__

'0.6.5.post73.dev0+g9cb5963'

In [3]:
ams.config_logger(stream_level=10)

In [4]:
sp = ams.load(ams.get_case('ieee14/ieee14_uc.xlsx'),
              setup=True,
              )

Input format guessed as xlsx.
Parsing input file "/home/jwang/work/ams/ams/cases/ieee14/ieee14_uc.xlsx"...
Input file parsed in 0.2663 seconds.
Adjust bus index to start from 0.
System set up in 0.0119 seconds.


In [5]:
sp.TimeSlot.n

6

In [6]:
sp.ED.setup(show_code=True)

ED data check passed.
- Generating symbols for ED
Set constrs pb: sum(pd1) + sum(pd2) - sum(pg)
Code Constr: om.constrs["pb"]=cp.sum(self.om.rtn.pd1.v) + cp.sum(self.om.rtn.pd2.v) - cp.sum(self.om.pg) == 0
Set constrs lub: PTDF1 @ (pg - pd1) - PTDF2 * pd2 - rate_a
Code Constr: om.constrs["lub"]=self.om.rtn.PTDF1.v @ (self.om.pg - self.om.rtn.pd1.v) - self.om.rtn.PTDF2.v @ self.om.rtn.pd2.v - self.om.rtn.rate_a.v <= 0


ValueError: Cannot broadcast dimensions  (5, 6) (5,)

In [ ]:
sp.UC.syms.generate_symbols()

In [ ]:
sp.UC.setup(show_code=True)

In [ ]:
sp.UC.run()

In [ ]:
sp.UC.obj.e_str

In [ ]:
sp.UC.ug.shape